In [17]:
from nltk.corpus import state_union
import bokeh

In [18]:
FILE = "/Users/zcarwile/nltk_data/corpora/state_union/1945-Truman.txt"

### Read document

In [19]:
with open(FILE,'r') as f:
    for line in f:
        for word in line.split(" "):
            print(word)

PRESIDENT
HARRY
S.
TRUMAN'S
ADDRESS
BEFORE
A
JOINT
SESSION
OF
THE
CONGRESS




April
16,
1945



Mr.
Speaker,
Mr.
President,
Members
of
the
Congress:

It
is
with
a
heavy
heart
that
I
stand
before
you,
my
friends
and
colleagues,
in
the
Congress
of
the
United
States.

Only
yesterday,
we
laid
to
rest
the
mortal
remains
of
our
beloved
President,
Franklin
Delano
Roosevelt.
At
a
time
like
this,
words
are
inadequate.
The
most
eloquent
tribute
would
be
a
reverent
silence.

Yet,
in
this
decisive
hour,
when
world
events
are
moving
so
rapidly,
our
silence
might
be
misunderstood
and
might
give
comfort
to
our
enemies.

In
His
infinite
wisdom,
Almighty
God
has
seen
fit
to
take
from
us
a
great
man
who
loved,
and
was
beloved
by,
all
humanity.

No
man
could
possibly
fill
the
tremendous
void
left
by
the
passing
of
that
noble
soul.
No
words
can
ease
the
aching
hearts
of
untold
millions
of
every
race,
creed
and
color.
The
world
knows
it
has
lost
a
heroic
champion
of
justice
and
freedom.

Tragic
fate
has
t

In [14]:
gwb_words = nltk.corpus.state_union.words('2006-GWBush.txt')

In [15]:
len(gwb_words)

6515

### Index document

In [20]:
nltk.corpus.state_union.words()

[u'PRESIDENT', u'HARRY', u'S', u'.', u'TRUMAN', u"'", ...]

In [21]:
nltk.corpus.state_union.fileids()

[u'1945-Truman.txt',
 u'1946-Truman.txt',
 u'1947-Truman.txt',
 u'1948-Truman.txt',
 u'1949-Truman.txt',
 u'1950-Truman.txt',
 u'1951-Truman.txt',
 u'1953-Eisenhower.txt',
 u'1954-Eisenhower.txt',
 u'1955-Eisenhower.txt',
 u'1956-Eisenhower.txt',
 u'1957-Eisenhower.txt',
 u'1958-Eisenhower.txt',
 u'1959-Eisenhower.txt',
 u'1960-Eisenhower.txt',
 u'1961-Kennedy.txt',
 u'1962-Kennedy.txt',
 u'1963-Johnson.txt',
 u'1963-Kennedy.txt',
 u'1964-Johnson.txt',
 u'1965-Johnson-1.txt',
 u'1965-Johnson-2.txt',
 u'1966-Johnson.txt',
 u'1967-Johnson.txt',
 u'1968-Johnson.txt',
 u'1969-Johnson.txt',
 u'1970-Nixon.txt',
 u'1971-Nixon.txt',
 u'1972-Nixon.txt',
 u'1973-Nixon.txt',
 u'1974-Nixon.txt',
 u'1975-Ford.txt',
 u'1976-Ford.txt',
 u'1977-Ford.txt',
 u'1978-Carter.txt',
 u'1979-Carter.txt',
 u'1980-Carter.txt',
 u'1981-Reagan.txt',
 u'1982-Reagan.txt',
 u'1983-Reagan.txt',
 u'1984-Reagan.txt',
 u'1985-Reagan.txt',
 u'1986-Reagan.txt',
 u'1987-Reagan.txt',
 u'1988-Reagan.txt',
 u'1989-Bush.txt',


### Document statistics